In [3]:
import matplotlib.pyplot as plt
import numpy as np
import librosa
import IPython
import numpy as np
import soundfile as sf
import torchaudio.functional as F
import torch
import os

In [91]:
audio_len = 22050 * 5
audio_len

110250

In [117]:
sample_rate = 22050

# read the wav file
# speech, sr = librosa.load(
#     "/data/ephraim/datasets/DNS-Challenge/synth1/clean/book_09145_chp_0006_reader_06390_2_door_Freesound_validated_212081_3_snr-10_fileid_9.wav"
# )
speech, sr = torchaudio.load(
    "/data/ephraim/datasets/DNS-Challenge/synth1/clean/book_09145_chp_0006_reader_06390_2_door_Freesound_validated_212081_3_snr-10_fileid_9.wav"
)
if sr != sample_rate:
    speech = F.resample(speech, sr, sample_rate)

In [118]:
speech = speech[:, :audio_len]
# speech = torch.clamp(speech[0], -1.0, 1.0)


# add gaussian noise
# noisy_wav = wav + np.random.normal(mean_, std_, wav.shape)
noise = torch.randn(1, speech.shape[1])

In [119]:
speech.shape

torch.Size([1, 110250])

In [120]:
noise.shape

torch.Size([1, 110250])

In [121]:
noise.shape[0]

1

In [122]:
# noise = torch.squeeze(noise)
# if noise.shape[0] < speech.shape[0]:
#     noise = np.pad(noise, ((0, speech.shape[0] - noise.shape[0])))
# else:
#     noise = noise[: speech.shape[0]]
snr = 3
noise_power = 1 / noise.shape[1] * torch.sum(noise**2)
speech_power = 1 / speech.shape[1] * torch.sum(speech**2)
noise_power_target = speech_power * np.power(10, -snr / 10)
noise_scaling = torch.sqrt(noise_power_target / noise_power)
lossy_speech = speech + noise_scaling * noise

In [123]:
noise_var = noise_scaling**2
noise_var

tensor(5.6738e-05)

In [124]:
torch.std(speech)**2

tensor(0.0001)

In [126]:
if torch.max(lossy_speech) >= 1 or torch.min(lossy_speech) <= -1:
    print("attention!")

In [127]:
from IPython.display import Audio, display

display(Audio(lossy_speech, rate=sample_rate, autoplay=False))

In [128]:
display(Audio(speech, rate=sample_rate, autoplay=False))

In [129]:
filename = "fileid_9_snr{}_var{}.wav".format(snr, noise_var)
filename

'fileid_9_snr3_var5.673767373082228e-05.wav'

In [23]:
# sf.write(
#     os.path.join("/data/ephraim/datasets/known_noise/clean_wav/", filename),
#     speech,
#     sr,
# )

In [130]:

tarpath

'/data/ephraim/datasets/known_noise/clean_wav/fileid_9_snr3_var5.673767373082228e-05.wav'

In [148]:
import torchaudio
import os

tarpath = os.path.join("/data/ephraim/datasets/known_noise/clean_wav/", filename)
torchaudio.save(tarpath, speech, sample_rate)

tar_noisypath = os.path.join("/data/ephraim/datasets/known_noise/noisy_wav/", filename)
torchaudio.save(tar_noisypath, lossy_speech, sample_rate)

In [142]:
speech2, sr = torchaudio.load(tarpath)

In [143]:
display(Audio(speech2, rate=sample_rate, autoplay=False))

In [144]:
display(Audio(speech, rate=sample_rate, autoplay=False))

In [145]:
torch.std(speech)

tensor(0.0106)

In [146]:
torch.std(speech2)

tensor(0.0106)

In [147]:
lossy_speech - lossy_speech2

tensor([[ 3.6152e-06, -6.0545e-06,  1.1676e-05,  ..., -1.1981e-06,
         -7.6226e-06,  7.0618e-06]])

In [149]:
specpath = r"fileid_9_snr3_var5.673767373082228e-05.wav"
if specpath.__contains__("var"):
    cur_noise_var = float(specpath.split("var")[1].split(".wav")[0])
cur_noise_var

5.673767373082228e-05